<a href="https://colab.research.google.com/github/julianVelandia/RedesNeuronalesConPyTorch/blob/master/redes-neuronales/3_3_Proyecto_de_redes_neuronales_FULL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de Precios de Casas utilizando el Conjunto de Datos de Boston Housing

Este proyecto construye, entrena y evalúa un modelo de red neuronal para predecir los precios de casas utilizando el conjunto de datos de Boston Housing.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np


### **Descripción de las Características del Conjunto de Datos Boston Housing**

| Índice | Característica | Descripción                                                                 |
|--------|----------------|-----------------------------------------------------------------------------|
| 0      | CRIM           | Tasa de criminalidad per cápita por localidad.                               |
| 1      | ZN             | Proporción de terreno residencial dividido en zonas para lotes de más de 25,000 pies cuadrados. |
| 2      | INDUS          | Proporción de acres de negocios no minoristas por ciudad.                    |
| 3      | CHAS           | Variable ficticia del Río Charles (1 si limita con el río; 0 de lo contrario).|
| 4      | NOX            | Concentración de óxidos nítricos (partes por 10 millones).                   |
| 5      | RM             | Número promedio de habitaciones por vivienda.                                |
| 6      | AGE            | Proporción de unidades ocupadas por propietarios construidas antes de 1940.  |
| 7      | DIS            | Distancias ponderadas a cinco centros de empleo en Boston.                   |
| 8      | RAD            | Índice de accesibilidad a carreteras radiales.                               |
| 9      | TAX            | Tasa de impuesto a la propiedad por cada $10,000.                            |
| 10     | PTRATIO        | Proporción alumno-maestro por ciudad.                                        |
| 11     | B              | 1000(Bk - 0.63)^2 donde Bk es la proporción de negros por ciudad.            |
| 12     | LSTAT          | Porcentaje de estatus inferior de la población.                              |



En esta sección, cargamos el conjunto de datos de Boston Housing, lo dividimos en conjuntos de entrenamiento y prueba, escalamos las características para mejorar el rendimiento de la red neuronal, y convertimos los datos en tensores de PyTorch.


In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

print("Datos originales cargados desde el archivo:\n")
print(raw_df.head(), "\n")

# Combina filas pares completas con las dos primeras columnas de las filas impares para formar X.
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])

# Extrae la tercera columna de las filas impares como etiquetas (y).
y = raw_df.values[1::2, 2]


print("Características (X) después de la transformación:\n")
print(X[:5], "\n")

print("Etiquetas (y) después de la transformación:\n")
print(y[:5], "\n")

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

print("Características como tensores de PyTorch (X_tensor):\n")
print(X_tensor[:5], "\n")

print("Etiquetas como tensores de PyTorch (y_tensor):\n")
print(y_tensor[:5], "\n")


Datos originales cargados desde el archivo:

          0      1      2    3      4      5     6       7    8      9     10
0    0.00632  18.00   2.31  0.0  0.538  6.575  65.2  4.0900  1.0  296.0  15.3
1  396.90000   4.98  24.00  NaN    NaN    NaN   NaN     NaN  NaN    NaN   NaN
2    0.02731   0.00   7.07  0.0  0.469  6.421  78.9  4.9671  2.0  242.0  17.8
3  396.90000   9.14  21.60  NaN    NaN    NaN   NaN     NaN  NaN    NaN   NaN
4    0.02729   0.00   7.07  0.0  0.469  7.185  61.1  4.9671  2.0  242.0  17.8 

Características (X) después de la transformación:

[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 7.1850e+00
  6.1100e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9283e+0

In [ ]:
def train_test_split(X, y, test_size=0.2):
    """
    Divide los datos en conjuntos de entrenamiento y prueba.
    """
    indices = torch.randperm(X.size(0))
    split = int(test_size * X.size(0))
    train_indices, test_indices = indices[split:], indices[:split]

    train_indices = indices[split:]
    test_indices = indices[:split]

    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = train_test_split(X_tensor, y_tensor)

print("Conjunto de entrenamiento (X_train_tensor):\n")
print(X_train_tensor[:5], "\n")
print("Conjunto de prueba (X_test_tensor):\n")
print(X_test_tensor[:5], "\n")
print("Etiquetas de entrenamiento (y_train_tensor):\n")
print(y_train_tensor[:5], "\n")
print("Etiquetas de prueba (y_test_tensor):\n")
print(y_test_tensor[:5], "\n")

# normalización
mean = X_train_tensor.mean(dim=0, keepdim=True)
std = X_train_tensor.std(dim=0, keepdim=True)

X_train_tensor = (X_train_tensor - mean) / std
X_test_tensor = (X_test_tensor - mean) / std

print("Conjunto de entrenamiento escalado (X_train_tensor):\n")
print(X_train_tensor[:5], "\n")
print("Conjunto de prueba escalado (X_test_tensor):\n")
print(X_test_tensor[:5], "\n")


Conjunto de entrenamiento (X_train_tensor):

tensor([[3.7050e-02, 2.0000e+01, 3.3300e+00, 0.0000e+00, 4.4290e-01, 6.9680e+00,
         3.7200e+01, 5.2447e+00, 5.0000e+00, 2.1600e+02, 1.4900e+01, 3.9223e+02,
         4.5900e+00],
        [6.1290e-02, 2.0000e+01, 3.3300e+00, 1.0000e+00, 4.4290e-01, 7.6450e+00,
         4.9700e+01, 5.2119e+00, 5.0000e+00, 2.1600e+02, 1.4900e+01, 3.7707e+02,
         3.0100e+00],
        [1.3075e+01, 0.0000e+00, 1.8100e+01, 0.0000e+00, 5.8000e-01, 5.7130e+00,
         5.6700e+01, 2.8237e+00, 2.4000e+01, 6.6600e+02, 2.0200e+01, 3.9690e+02,
         1.4760e+01],
        [1.7446e-01, 0.0000e+00, 1.0590e+01, 1.0000e+00, 4.8900e-01, 5.9600e+00,
         9.2100e+01, 3.8771e+00, 4.0000e+00, 2.7700e+02, 1.8600e+01, 3.9325e+02,
         1.7270e+01],
        [1.6128e+00, 0.0000e+00, 8.1400e+00, 0.0000e+00, 5.3800e-01, 6.0960e+00,
         9.6900e+01, 3.7598e+00, 4.0000e+00, 3.0700e+02, 2.1000e+01, 2.4831e+02,
         2.0340e+01]]) 

Conjunto de prueba (X_test_tenso


Aquí definimos una red neuronal con dos capas ocultas para predecir el precio de las casas. Utilizamos funciones de activación ReLU para las capas ocultas.


In [ ]:
class HousePricePredictor(nn.Module):
    """
    Red neuronal multicapa para predecir precios de casas, con tres capas
    totalmente conectadas y funciones de activación ReLU en las dos primeras capas.
    """
    def __init__(self, input_dim):
        """
        Inicializa las capas completamente conectadas de la red neuronal.
        """
        super(HousePricePredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        """
        Se define el paso hacia adelante de la red, aplicando ReLU a las dos primeras capas.
        """
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = X_train_tensor.shape[1]
model = HousePricePredictor(input_dim)
print(f"Modelo de red neuronal creado: {model}")


Modelo de red neuronal creado: HousePricePredictor(
  (fc1): Linear(in_features=13, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)



Definimos la función de pérdida utilizando el error cuadrático medio (MSE) y usamos el optimizador Adam para actualizar los pesos de la red durante el entrenamiento.


In [ ]:
# Función de pérdida
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)



Entrenamos el modelo utilizando el conjunto de datos de entrenamiento. La pérdida se imprime cada 10 épocas para monitorizar el progreso.


In [ ]:

def train_model(model, criterion, optimizer, X_train, y_train, epochs=100):
    """
    Entrena el modelo durante un número específico de épocas utilizando el conjunto de datos de entrenamiento.
    """

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor)


Epoch 1/100, Loss: 597.7412109375
Epoch 2/100, Loss: 585.080810546875
Epoch 3/100, Loss: 570.7589721679688
Epoch 4/100, Loss: 552.3079223632812
Epoch 5/100, Loss: 528.4374389648438
Epoch 6/100, Loss: 498.5988464355469
Epoch 7/100, Loss: 462.3499755859375
Epoch 8/100, Loss: 419.6030578613281
Epoch 9/100, Loss: 370.7037048339844
Epoch 10/100, Loss: 316.71270751953125
Epoch 11/100, Loss: 259.7393493652344
Epoch 12/100, Loss: 203.23452758789062
Epoch 13/100, Loss: 152.25320434570312
Epoch 14/100, Loss: 113.53994750976562
Epoch 15/100, Loss: 94.09928894042969
Epoch 16/100, Loss: 97.07152557373047
Epoch 17/100, Loss: 115.6932373046875
Epoch 18/100, Loss: 133.18214416503906
Epoch 19/100, Loss: 135.74977111816406
Epoch 20/100, Loss: 122.01611328125
Epoch 21/100, Loss: 98.84598541259766
Epoch 22/100, Loss: 74.45112609863281
Epoch 23/100, Loss: 54.78443145751953
Epoch 24/100, Loss: 42.3873405456543
Epoch 25/100, Loss: 36.91046905517578
Epoch 26/100, Loss: 36.37610626220703
Epoch 27/100, Loss: 38


Evaluamos el rendimiento del modelo en el conjunto de datos de prueba y calculamos el error cuadrático medio.


In [ ]:

def evaluate_model(model, X_test, y_test):
    """
    Evalúa el rendimiento del modelo en el conjunto de datos de prueba y calcula el error cuadrático medio.
    """
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        loss = criterion(predictions, y_test)
    print(f"Error Cuadrático Medio en el conjunto de prueba: {loss.item()}")

evaluate_model(model, X_test_tensor, y_test_tensor)


Error Cuadrático Medio en el conjunto de prueba: 22.647968292236328



Simulamos las características de una nueva casa, las escalamos utilizando el mismo escalador que se usó en los datos de entrenamiento, y luego predecimos el precio de la casa utilizando el modelo entrenado.


In [ ]:
mean = X_train_tensor.mean(dim=0, keepdim=True)
std = X_train_tensor.std(dim=0, keepdim=True)

new_house = torch.tensor([[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.0900, 1.0, 296.0, 15.3, 396.90, 4.98]], dtype=torch.float32)

new_house_scaled = (new_house - mean) / std

model.eval()
with torch.no_grad():
    predicted_price = model(new_house_scaled)
print(f"Predicción del precio de la nueva casa: ${predicted_price.item() * 1000:.2f}")


Predicción del precio de la nueva casa: $1643290.41


| Índice | Característica | Valor  | Descripción                                                                 | Evaluación          |
|--------|----------------|--------|-----------------------------------------------------------------------------|---------------------|
| 0      | CRIM           | 0.00632 | Tasa de criminalidad per cápita por localidad.                               | Favorable (bajo)    |
| 1      | ZN             | 18.0    | Proporción de terreno residencial dividido en zonas para lotes de más de 25,000 pies cuadrados. | Favorable           |
| 2      | INDUS          | 2.31    | Proporción de acres de negocios no minoristas por ciudad.                    | Favorable (bajo)    |
| 3      | CHAS           | 0.0     | Variable ficticia del Río Charles (1 si limita con el río; 0 de lo contrario).| No favorable        |
| 4      | NOX            | 0.538   | Concentración de óxidos nítricos (partes por 10 millones).                   | Favorable (bajo)    |
| 5      | RM             | 6.575   | Número promedio de habitaciones por vivienda.                                | Favorable           |
| 6      | AGE            | 65.2    | Proporción de unidades ocupadas por propietarios construidas antes de 1940.  | No favorable (alto) |
| 7      | DIS            | 4.0900  | Distancias ponderadas a cinco centros de empleo en Boston.                   | Favorable           |
| 8      | RAD            | 1.0     | Índice de accesibilidad a carreteras radiales.                               | Favorable           |
| 9      | TAX            | 296.0   | Tasa de impuesto a la propiedad por cada $10,000.                            | Favorable (bajo)    |
| 10     | PTRATIO        | 15.3    | Proporción alumno-maestro por ciudad.                                        | Favorable (bajo)    |
| 11     | B              | 396.90  | 1000(Bk - 0.63)^2 donde Bk es la proporción de negros por ciudad.            | Favorable           |
| 12     | LSTAT          | 4.98    | Porcentaje de estatus inferior de la población.                              | Favorable (bajo)    |
